# Ring spot helper demonstration

This notebook demonstrates the analytic ring spot helper using the same mesh API that powers `add_spherical_harmonic_spot` or `add_pulsation`, so you can drop it into the usual SPICE surface modeling workflow.

In [ ]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from spice.models import IcosphereModel
from spice.models.spots import add_ring_spot, RingSpotConfig
from spice.utils.ring_spot import ring_spot_weights, ca_irt_scale_map, apply_ca_irt_scaling_local


## Build a base mesh and apply `add_ring_spot`

In [ ]:
cfg = RingSpotConfig()
base_temp = 5700.0
base_ca = -0.1

mesh = IcosphereModel.construct(
    2000,
    1.0,
    1.0,
    jnp.array([base_temp, base_ca]),
    ['teff', 'ca']
)

spot_axis = jnp.array([0.0, 0.0, 1.0])
spotted_mesh = add_ring_spot(
    mesh,
    param_index=0,
    config=cfg,
    ca_param_index=1,
    spot_axis=spot_axis,
)

normals = mesh.d_centers / jnp.linalg.norm(mesh.d_centers, axis=1, keepdims=True)
w_umb, w_plage = ring_spot_weights(normals, spot_axis, cfg)

normals_np = np.array(normals)
lon = np.degrees(np.arctan2(normals_np[:, 1], normals_np[:, 0]))
lat = np.degrees(np.arcsin(normals_np[:, 2]))

temp_delta = np.array(spotted_mesh.parameters[:, 0] - mesh.parameters[:, 0])
ca_delta = np.array(spotted_mesh.parameters[:, 1] - mesh.parameters[:, 1])


### Umbra and plage weights

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=np.array(w_umb), s=10, cmap='magma', vmin=0.0)
axes[0].set_title('Umbra weight')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='w_umb')

sc1 = axes[1].scatter(lon, lat, c=np.array(w_plage), s=10, cmap='viridis', vmin=0.0)
axes[1].set_title('Plage ring weight')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='w_plage')

plt.show()


### Parameter perturbations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=temp_delta, s=10, cmap='coolwarm')
axes[0].set_title('Temperature perturbation [K]')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='ΔT')

sc1 = axes[1].scatter(lon, lat, c=ca_delta, s=10, cmap='Spectral')
axes[1].set_title('Ca abundance perturbation [dex]')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='Δ[Ca/H]')

plt.show()


## Ca II IRT scaling factors

In [ ]:
scale_map = ca_irt_scale_map(normals, spot_axis, cfg)

fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
axes[0].hist(np.array(scale_map), bins=30, color='tab:blue')
axes[0].set_xlabel('Scale factor')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Ca II IRT scaling')

sc = axes[1].scatter(lon, lat, c=np.array(scale_map), s=10, cmap='plasma')
axes[1].set_title('Spatial map of scaling')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc, ax=axes[1], label='S_CaIRT')

plt.show()


## Applying the scaling to a toy line profile

In [ ]:
wavelength = jnp.linspace(848.0, 870.0, 800)
I_cont = jnp.ones_like(wavelength)
I_lambda = I_cont - 0.45 * jnp.exp(-0.5 * ((wavelength - 854.2) / 0.45) ** 2)
mask_irt = (wavelength > 852.5) & (wavelength < 856.0)

scale_example = float(scale_map[np.argmax(np.array(w_plage))])
I_scaled = apply_ca_irt_scaling_local(I_lambda, I_cont, scale_example, mask_irt)

plt.figure(figsize=(8, 4))
plt.plot(wavelength, I_lambda, label='Original profile')
plt.plot(wavelength, I_scaled, label=f'Scaled (S={scale_example:.2f})')
plt.xlabel('Wavelength [nm]')
plt.ylabel('Normalized intensity')
plt.title('Ca II IRT depth scaling example')
plt.legend()
plt.show()
